In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_california_housing
california_housing  = fetch_california_housing(as_frame=True)


df = california_housing.frame 
feature_list = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Longitude', 'Latitude']
target = 'MedHouseVal' 
y = df[target].to_numpy()
X = df[feature_list].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
df.head()
len(df)

20640

#### MinMaxScalar is to scale data as follows:
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) 

X_scaled = X_std * (max - min) + min

In [2]:
# use minMax scaler
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

In [12]:
# build the model!
model = Sequential()
model.add(Dense(1000, input_shape=(X_train.shape[1],), activation='relu')) # (features,)
model.add(Dropout(0.5)) # specify a percentage between 0 and 0.5, or larger
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5)) # specify a percentage between 0 and 0.5, or larger
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5)) # specify a percentage between 0 and 0.5, or larger
model.add(Dense(1, activation='linear')) # output node
model.summary() # see what your model looks like

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1000)              9000      
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 1000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               500500    
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 500)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 250)               125250    
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 250)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [13]:
# compile the model
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

# early stopping callback
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=50,
                   restore_best_weights = True)

# fit the model!
# attach it to a new variable called 'history' in case
# to look at the learning curves
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    callbacks=[es],
                    epochs=100,
                    batch_size=200,
                    verbose=1)

Epoch 1/100
83/83 [==============================] - 3s 21ms/step - loss: 1.0472 - mae: 0.7647 - val_loss: 0.6025 - val_mae: 0.5651
Epoch 2/100
83/83 [==============================] - 2s 21ms/step - loss: 0.7824 - mae: 0.6498 - val_loss: 0.5648 - val_mae: 0.5520
Epoch 3/100
83/83 [==============================] - 2s 21ms/step - loss: 0.7119 - mae: 0.6139 - val_loss: 0.5666 - val_mae: 0.5095
Epoch 4/100
83/83 [==============================] - 2s 20ms/step - loss: 0.6726 - mae: 0.5971 - val_loss: 0.5963 - val_mae: 0.5121
Epoch 5/100
83/83 [==============================] - 2s 20ms/step - loss: 0.6375 - mae: 0.5762 - val_loss: 0.5168 - val_mae: 0.5495
Epoch 6/100
83/83 [==============================] - 2s 20ms/step - loss: 0.6084 - mae: 0.5635 - val_loss: 0.5483 - val_mae: 0.5738
Epoch 7/100
83/83 [==============================] - 2s 20ms/step - loss: 0.5935 - mae: 0.5572 - val_loss: 0.4894 - val_mae: 0.4725
Epoch 8/100
83/83 [==============================] - 2s 20ms/step - loss: 0.

In [14]:
pred = model.predict(X_test)
from sklearn.metrics import r2_score
print('R2 Value:', r2_score(y_test, pred)) # test

R2 Value: 0.7001273553082866


In [15]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 10, max_depth=15, min_samples_leaf = 15, random_state = 0)
model_rf = rf.fit(X_train, y_train)
prediction=model_rf.predict(X_test)
print('R2 Value:',r2_score(y_test, prediction))

R2 Value: 0.7707653024039927
